In [26]:
import tensorflow as tf
import sentencepiece as spm
import numpy as np
import pickle
import gzip

from tqdm import tqdm

In [27]:
path = '/hdd/user16/HT/'

spm_model = './30k-clean.model'
sp = spm.SentencePieceProcessor()
sp.load(spm_model)

True

In [28]:
max_seq = 256

In [29]:
pad = sp.piece_to_id('<pad>')
unk = sp.piece_to_id('<unk>')
CLS = sp.piece_to_id('[CLS]')
SEP = sp.piece_to_id('[SEP]')
MASK = sp.piece_to_id('[MASK]')

In [34]:
#TODO: NSP 만들고, 레이블 생성
#file_names = ['xaa', 'xab', 'xac', 'xad', 'xae', 'xaf', 'xag']
file_names = ['xab', 'xac', 'xad', 'xae', 'xaf', 'xag']
files = [path+x for x in file_names]

# read data

for idx, file in enumerate(files):
    nsp = []
    nsp_label = []

    sep_positions = []
    sep_zeros = [0 for _ in range(max_seq)]
    sep_ones = [1 for _ in range(max_seq)]

    masks = []
    pads = [0 for _ in range(max_seq)]

    with open(file, 'r') as f:
        current_head = 1e9
        prev_line = ''
        for i, x in enumerate(tqdm(f)):
            if i == (current_head+1):
                continue
            if x == '\n':
                current_head = i
                
            line = x[:-1]
            line = sp.EncodeAsIds(line)            
            
            if i == 0:
                prev_line = line
            else:
                if len(line) + len(prev_line) <= max_seq - 2:
                    if np.random.rand(1) < 0.5:
                        tmp = [CLS] +  line + [SEP] + prev_line
                        nsp_label.append(1)
                    else:
                        tmp = [CLS] +  prev_line + [SEP] + line
                        nsp_label.append(0)
                        
                    masks.append(len(tmp))
                    
                    tmp = tmp + pads[len(tmp):]
                    nsp.append(tmp)                 
                    
                    sep_position = tmp.index(SEP)                 
                    sep_positions.append(sep_position)
                prev_line = line
        
        nsp = np.array(nsp)
        nsp_label = np.array(nsp_label)
        sep_positions = np.array(sep_positions)
        masks = np.array(masks)
        
        print(nsp.shape, nsp_label.shape, sep_positions.shape, masks.shape)
        
        with open(f'./data/{file_names[idx]}.pickle', 'wb') as ff:
            pickle.dump(nsp, ff, protocol=4)
            pickle.dump(nsp_label, ff, protocol=4)
            pickle.dump(sep_positions, ff, protocol=4)
            pickle.dump(masks, ff, protocol=4)

10000000it [15:16, 10914.00it/s]


(7583456, 256) (7583456,) (7583456,) (7583456,)


10000000it [13:48, 12073.51it/s]


(7678516, 256) (7678516,) (7678516,) (7678516,)


10000000it [13:14, 12586.39it/s]


...

(7607751, 256) (7607751,) (7607751,) (7607751,)


10000000it [11:47, 14127.47it/s]


(7698200, 256) (7698200,) (7698200,) (7698200,)


6561002it [07:19, 14928.28it/s]


(5062123, 256) (5062123,) (5062123,) (5062123,)


In [32]:
with gzip.open(f'./data/{file_names[idx]}.pickle', 'wb') as f:
    pickle.dump(nsp, f, protocol=4)
    pickle.dump(nsp_label, f, protocol=4)
    pickle.dump(sep_positions, f, protocol=4)
    pickle.dump(masks, f, protocol=4)

In [25]:
np.random.randn(10,10)

array([[-0.78494951, -1.19990029,  0.39980795, -0.20708565, -0.25139225,
         0.63747917, -1.08715843,  0.09835226, -0.55893874, -1.32569713],
       [-2.28131739, -1.28306348,  0.46355663,  0.60456615, -0.25912351,
        -1.92884701,  1.30761999,  0.3595043 , -2.04121625,  0.53387862],
       [-0.71177076, -0.82135535,  0.35293037, -1.70557325, -0.23503857,
         0.06883204,  1.52503056,  0.58358535,  1.2631964 ,  1.60097638],
       [-1.0383294 ,  0.06083734, -1.21641578, -0.61748875, -0.21670945,
         0.19150848,  0.29454754,  1.34198555, -0.09593865, -0.38558859],
       [-0.54573941,  1.45375845,  0.23842658,  1.27977178, -0.58890031,
         1.19754331,  1.92078301, -0.26031847,  0.55212242, -1.13969608],
       [-0.98732033,  0.1863012 , -1.25211218,  0.00256408,  1.23306526,
         0.29224557,  0.16778989, -1.18349694, -0.7401791 , -1.67392392],
       [-0.0935759 , -1.95255944,  0.57663495,  0.44791249, -0.1300575 ,
        -0.76260954, -0.52650226,  0.65557747

In [22]:
tf.__version__

'1.15.3'